## Resampling with GDAL's Python API (Local storage, NetCDF File, NetCDF4 driver)

In [ ]:
import argparse
import sys

from osgeo import gdal
from pyproj.crs import CRS

In [ ]:
sys.path.append("..")
gdal.UseExceptions()

In [ ]:
def warp_resample(dataset):
    from common import earthaccess_args
    from common import target_extent as te

    # Define filepath, driver, and variable information
    args = earthaccess_args[dataset]
    src = f'NETCDF:earthaccess_data/{args["filename"]}:{args["variable"]}'
    # Set a memory file as the output format
    output = ""
    output_format = "MEM"
    # Set web mercator as the output projection
    dstSRS = "EPSG:3857"
    # Define a geotransform for the output tile
    width = height = 256
    gt = [
        te[0],
        (te[2] - te[0]) / width,
        0,
        te[3],
        0,
        -(te[3] - te[1]) / height,
    ]
    output_crs = CRS(dstSRS).to_wkt()
    # Create a GDAL dataset to store the output
    output_ds = gdal.GetDriverByName(output_format).Create(
        output, width, height, 1, gdal.GDT_Byte
    )
    output_ds.SetProjection(output_crs)
    output_ds.SetGeoTransform(gt)
    # Open the input dataset
    input_ds = gdal.Open(src)
    # Warp transform the input dataset
    return gdal.Warp(output_ds, input_ds)

In [ ]:
if __name__ == "__main__":
    if "get_ipython" in dir():
        # Just call warp_resample if running as a Jupyter Notebook
        da = warp_resample("gpm_imerg")
    else:
        # Configure dataset via argpase if running via CLI
        parser = argparse.ArgumentParser(description="Set environment for the script.")
        parser.add_argument(
            "--dataset",
            default="mursst",
            help="Dataset to resample.",
            choices=["mursst"],
        )
        user_args = parser.parse_args()
        da = warp_resample(user_args.dataset)